In [1]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [2]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [3]:
leagues_model.load_counties_data()

### Blank Sandbox

In [4]:
leagues_model.reset_county_styles()

leagues_model.render_map(None)

HTML(value='\n            <style>\n            .leaflet-popup-content-wrapper .leaflet-popup-tip {\n          …

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
from sportwarz import LeagueJson, LeaguesModel, league_teams_sums

import ast
import json
import codecs
import ipywidgets as widgets
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_rows', 256)
pd.set_option('display.width', 256)

leagues = { }

team_league_select = widgets.Dropdown(description = "League:")
add_button = widgets.Button(description="Add Team")

error_message = widgets.Label(style = { "color" : "red" })

out = widgets.Output()
out.clear_output()

load_league_input = widgets.Dropdown(description = "League to Load:",
                                      options = ["NFL", "MLB", "NBA", "NHL", "MLS"],
                                      style={'description_width': '120px'})
load_league_button = widgets.Button(description="Load League")

upload = widgets.FileUpload(
    description="Upload JSON",
    accept='.json', 
    multiple= False  
)

team_league_select = widgets.Dropdown(description = "League:", style={'description_width': '60px'})
team_select = widgets.SelectMultiple(description = "Teams:", layout={'height': '100px'})
team_textarea = widgets.Textarea(value = "", description="JSON:", style={'description_width': '60px'}, layout=widgets.Layout(width="400px", height="300px"))
team_set_coordinates_state_button = widgets.Button(
    description = "Use Last Popup Coordinates/State",
    layout=widgets.Layout(width="250px"))
add_update_team_button = widgets.Button(description = "Add/Update Team(s)")
delete_add_update_team_button = widgets.Button(
    description = "Delete Team + Add/Update Team(s)",
    layout=widgets.Layout(width="250px"))
delete_button = widgets.Button(description="Delete Teams")

eval_textarea = widgets.Textarea(value = "", description="Python Code:", style={'description_width': '60px'}, layout=widgets.Layout(width="100%", height="400px"))
eval_execute_button = widgets.Button(description="Execute Code")

load_team_panel = widgets.GridBox(children = [
    load_league_input, widgets.Label("Load JSON data stored on server"),
    load_league_button
], layout= widgets.Layout(
        grid_template_columns="repeat(2, 300px)",
        grid_gap="10px 10px")
)

upload_panel = widgets.GridBox(children = [
   upload, widgets.Label("Upload JSON file holding league data to server"),
], layout= widgets.Layout(
        grid_template_columns="150px 300px",
        grid_gap="10px 10px")
)

add_panel = widgets.GridBox(children = [
    team_league_select, widgets.Label("League to add/update/delete team"), 
    team_select, widgets.Label("Multi-select of team(s) to update/delete"), 
    delete_button, widgets.HTML(""),
    team_textarea, widgets.HTML("""
        <p>JSON list of teams to add</p>
        <table width="100%">
        </table>"""), 
    add_update_team_button, widgets.HTML(""),
    delete_add_update_team_button, widgets.HTML(""),
    team_set_coordinates_state_button, widgets.HTML("""
    Create JSON template with coordinates of the centroid of the last county a popup was displayed.""")
], layout= widgets.Layout(
        grid_template_columns="400px 600px",
        grid_gap="10px 10px")
)

eval_panel = widgets.VBox([
    eval_textarea,
    eval_execute_button
])

tab = widgets.Tab()
tab.children = [load_team_panel, upload_panel, add_panel, eval_panel]
tab.titles =   ["Load League", "Upload League", "Add/Update/Delete Teams", "Evaluate Python"]

league_dropdown_options = []

@out.capture()
def lock_tabs(locked=True, widget=tab):
    if hasattr(widget, "children"):
        for child in widget.children:
            lock_tabs(locked, child)
    elif hasattr(widget, "disabled"):
        widget.disabled = locked

@out.capture()
def show_pre_post_merged_results(league_name:str, before_model:LeaguesModel, after_model: LeaguesModel):
    pre_sums = league_teams_sums(before_model._leagues[league_name])
    post_sums = league_teams_sums(after_model._leagues[league_name])
    merged = pd.merge(pre_sums, post_sums, on='team_name', how='outer', suffixes=("_before", "_after"))
    merged["share_population_before"] = merged["share_population_before"].fillna(0) 
    merged["share_population_after"] = merged["share_population_after"].fillna(0) 
    merged["share_population_value_before"] = merged["share_population_value_before"].fillna(0) 
    merged["share_population_value_after"] = merged["share_population_value_after"].fillna(0) 
    merged["share_population_delta"] = merged["share_population_after"] - merged["share_population_before"]
    merged["share_population_value_delta"] = merged["share_population_value_after"] - merged["share_population_value_before"]
    print(merged)
    print(f'Population Values Sums\t: {merged["share_population_value_before"].sum():,.0f}\t{merged["share_population_value_after"].sum():,.0f}')
    print(f'Population Values Sum Delta = {(merged["share_population_value_after"].sum() - merged["share_population_value_before"].sum()):,.0f}')        

@out.capture()
def add_update_teams(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        if team_league_select.value:
            new_teams = json.loads(team_textarea.value)
            before_model = leagues_model.copy_with_just_league(team_league_select.value)

            leagues_model.add_teams(team_league_select.value, new_teams)
            leagues_model.calculate_distances() 
            leagues_model.compute_shares()  
            leagues_model.heatmap_counties(team_league_select.value)
            leagues_model.refresh_geojson_layer()
            show_pre_post_merged_results(team_league_select.value, before_model, leagues_model) 
            league_select_change(None)   
        else:
            error_message.value = "League not selected"
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)

@out.capture()
def last_click_template(button):
    try:
        team_textarea.value = f"""[{{
            "name": "Somewhere Whatevers",
            "venue": "Venue Arena Stadium",
            "L": 1.0,
            "S": 3.0,
            "N": 3.0,
            "coordinates": {{
                "lat": {leagues_model.last_click_lat:.3f},
                "lon": {leagues_model.last_click_lon:.3f}
            }},
            "color": "#001122",
            "state": "{leagues_model.last_click_state}"
    }}                                     
]"""
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False) 

@out.capture()
def delete_teams(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        if team_league_select.value:
            before_model = leagues_model.copy_with_just_league(team_league_select.value)
            if leagues_model.delete_teams(team_league_select.value, team_select.value):
                leagues_model.calculate_distances() 
                leagues_model.compute_shares()  
                leagues_model.heatmap_counties(team_league_select.value)
                leagues_model.refresh_geojson_layer()
                show_pre_post_merged_results(team_league_select.value, before_model, leagues_model)      
                league_select_change(None)
        else:
            error_message.value = "League not selected"
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)

@out.capture()
def league_select_change(change):
    team_select.options = sorted(set(map(lambda team: team["name"], leagues_model._leagues[team_league_select.value]["teams"])))

@out.capture()
def load_league(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        league_name = load_league_input.value 
        if not league_name in leagues_model._leagues:
            leagues_model.load_leagues([league_name])
            leagues_model.calculate_distances() 
            leagues_model.compute_shares()  
    
        leagues_model.heatmap_counties(league_name)
        leagues_model.refresh_geojson_layer()
      
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)  
            team_league_select.options = league_dropdown_options
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)


@out.capture()
def on_upload_change(change):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        file_info = change["new"][0]
        content = codecs.decode(file_info.content, encoding="utf-8")
        league_json: LeagueJson = json.loads(content)
        league_name = league_json["league_name"]
        leagues_model.add_league(league_json)
        leagues_model.calculate_distances() 
        leagues_model.compute_shares()  
    
        leagues_model.heatmap_counties(league_name)
        leagues_model.refresh_geojson_layer()
      
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)  
            team_league_select.options = league_dropdown_options
    except Exception as ex:
        print(ex)        
    finally:
        lock_tabs(False)

@out.capture()
def delete_add_update_team(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        if team_league_select.value:
            new_teams = json.loads(team_textarea.value)
            
            before_model = leagues_model.copy_with_just_league(team_league_select.value)
            leagues_model.delete_teams(team_league_select.value, team_select.value, False)
            leagues_model.add_teams(team_league_select.value, new_teams)

            leagues_model.heatmap_counties(team_league_select.value)
            leagues_model.refresh_geojson_layer()
            show_pre_post_merged_results(team_league_select.value, before_model, leagues_model)     
            league_select_change(None)
        else:
            error_message.value = "League not selected"
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)

@out.capture()
def eval_execute(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        ast.parse(eval_textarea.value, mode="exec")
        exec(eval_textarea.value, {}, {
            "leagues_model": leagues_model,
            "show_pre_post_merged_results": show_pre_post_merged_results
            })
    except SyntaxError as e:
        print("Syntax error at line", e.lineno, "col", e.offset)
        print(e.text)
    except Exception as e:
        print("Eval runtime error:", e)
    finally:
        lock_tabs(False)    

load_league_button.on_click(load_league)

upload.observe(on_upload_change, names="value")  

add_update_team_button.on_click(add_update_teams)
delete_add_update_team_button.on_click(delete_add_update_team)

team_set_coordinates_state_button.on_click(last_click_template)

team_league_select.observe(league_select_change, names='value')
delete_button.on_click(delete_teams) 

eval_execute_button.on_click(eval_execute)

widgets.VBox([
    tab,
    error_message
])


In [ ]:
out